In [26]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

In [27]:
train_ds = keras.utils.image_dataset_from_directory(
    directory='Alzheimer_s Dataset/train',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(256, 256))

Found 5121 files belonging to 4 classes.


In [28]:
test_ds = keras.utils.image_dataset_from_directory(
    directory = 'Alzheimer_s Dataset/test',
    labels = 'inferred',
    label_mode = 'categorical',
    batch_size = 32,
    image_size = (256, 256)
    )

Found 1279 files belonging to 4 classes.


In [29]:
print("Summary of the training dataset : ", train_ds)
print("Summary of the test dataset : ", test_ds)

Summary of the training dataset :  <BatchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))>
Summary of the test dataset :  <BatchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))>


In [30]:
from keras.applications.vgg16 import VGG16
VGG = VGG16(include_top=False, input_shape=(256,256,3),pooling='max')    

VGG.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [31]:
from tensorflow.keras import models, layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
model = keras.models.Sequential()

for layer in VGG.layers:
    layer.trainable=False
    
model.add(VGG)
model.add(Flatten())
model.add(Dense(512,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(128,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(32,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(4,activation='softmax'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 flatten_3 (Flatten)         (None, 512)               0         
                                                                 
 dense_12 (Dense)            (None, 512)               262656    
                                                                 
 dense_13 (Dense)            (None, 128)               65664     
                                                                 
 dense_14 (Dense)            (None, 32)                4128      
                                                                 
 dense_15 (Dense)            (None, 4)                 132       
                                                                 
Total params: 15,047,268
Trainable params: 332,580
Non

In [32]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),  
      tf.keras.metrics.AUC(name='auc')
]

In [34]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=METRICS)

In [36]:
model.fit(train_ds, epochs=30)

Epoch 1/30
161/161 [==============================] - 54s 330ms/step - loss: 0.1904 - accuracy: 0.9643 - precision: 0.9323 - recall: 0.9244 - auc: 0.9927
Epoch 2/30
161/161 [==============================] - 54s 334ms/step - loss: 0.1140 - accuracy: 0.9798 - precision: 0.9617 - recall: 0.9574 - auc: 0.9973
Epoch 3/30
161/161 [==============================] - 53s 331ms/step - loss: 0.0923 - accuracy: 0.9834 - precision: 0.9681 - recall: 0.9652 - auc: 0.9982
Epoch 4/30
 73/161 [============>.................] - ETA: 29s - loss: 0.5099 - accuracy: 0.9195 - precision: 0.8420 - recall: 0.8348 - auc: 0.9643

KeyboardInterrupt: 

### 98.52 for VGG16 for avg pooling at 14th epoch
### 98.34 for VGG16 for max pooling

In [37]:
EfficientNet = keras.applications.EfficientNetV2B0(
               include_top = False, input_shape = (256, 256, 3), pooling = 'max')

24274472/24274472 [==============================] - 41s 2us/step


In [38]:
model1 = keras.models.Sequential()
for layer in EfficientNet.layers:
    layer.trainable = False

model1.add(EfficientNet)
model1.add(Flatten())
model1.add(Dense(512,activation='relu',kernel_initializer='he_uniform'))
model1.add(Dense(128,activation='relu',kernel_initializer='he_uniform'))
model1.add(Dense(32,activation='relu',kernel_initializer='he_uniform'))
model1.add(Dense(4,activation='softmax'))

model1.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-b0 (Function  (None, 1280)             5919312   
 al)                                                             
                                                                 
 flatten_4 (Flatten)         (None, 1280)              0         
                                                                 
 dense_16 (Dense)            (None, 512)               655872    
                                                                 
 dense_17 (Dense)            (None, 128)               65664     
                                                                 
 dense_18 (Dense)            (None, 32)                4128      
                                                                 
 dense_19 (Dense)            (None, 4)                 132       
                                                      

In [39]:
model1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=METRICS)

In [40]:
model1.fit(train_ds, epochs = 50)

Epoch 1/50
161/161 [==============================] - 27s 119ms/step - loss: 1.1975 - accuracy: 0.8135 - precision: 0.6606 - recall: 0.5222 - auc: 0.8591
Epoch 2/50
161/161 [==============================] - 19s 115ms/step - loss: 0.9387 - accuracy: 0.7904 - precision: 0.6284 - recall: 0.3956 - auc: 0.8335
Epoch 3/50
161/161 [==============================] - 19s 115ms/step - loss: 0.8927 - accuracy: 0.7983 - precision: 0.6488 - recall: 0.4214 - auc: 0.8478
Epoch 4/50
161/161 [==============================] - 19s 115ms/step - loss: 0.8625 - accuracy: 0.8067 - precision: 0.6726 - recall: 0.4417 - auc: 0.8585
Epoch 5/50
161/161 [==============================] - 19s 116ms/step - loss: 0.8125 - accuracy: 0.8178 - precision: 0.6928 - recall: 0.4874 - auc: 0.8754
Epoch 6/50
161/161 [==============================] - 19s 116ms/step - loss: 0.8415 - accuracy: 0.8113 - precision: 0.6714 - recall: 0.4804 - auc: 0.8656
Epoch 7/50
161/161 [==============================] - 19s 118ms/step - loss:

### 91.15 for Efficient Net